In [268]:
import nltk
from nltk.corpus import brown
from nltk.corpus import stopwords
#from baiye import BayesianFilter
#bf = BayesianFilter()
import math, sys
from konlpy.tag import Twitter
import re

In [391]:
class BayesianFilter:
    def __init__(self):
        self.words = set() # 출현한 단어 기록
        self.word_dict = {} # 카테고리마다의 출현 횟수 기록
        self.category_dict = {} # 카테고리 출현 횟수 기록
        self.cfd = _
        
    # 형태소 분석하기 --- (※1)
    def split(self, text):
        return text.split()
        results = []
        twitter = Twitter()
        # 단어의 기본형 사용
        malist = twitter.pos(text, norm=True, stem=True)
        for word in malist:
            # 어미/조사/구두점 등은 대상에서 제외 
            if not word[1] in ["Josa", "Eomi", "Punctuation"]:
                results.append(word[0])
        return results
    
    def fit_cfd(self, cfd):
        
        for category in cfd.conditions():
            
            for word in cfd[category]:
                if not category in self.category_dict:
                    self.category_dict[category] = 0
                self.category_dict[category] += 1
                
                if not category in self.word_dict:
                    self.word_dict[category] = {}
                if not word in self.word_dict[category]:
                    self.word_dict[category][word] = 0
                self.word_dict[category][word] = cfd[category][word]
                self.words.add(word)

    def fit_list(self, data):
        
        training_data = []
        
        for c, w in data:
            if not c in self.category_dict:
                   self.category_dict[c] = 0
                    
            self.category_dict[c] += 1
            self.words.add(w)
            
            training_data += {(c,w)}
            
        self.cfd = nltk.ConditionalFreqDist(training_data)
        
    # 단어 리스트에 점수 매기기--- (※4)
    def score(self, words, category):
        s_category = sum(self.category_dict.values())         
        score = math.log( self.category_dict[category] / s_category )
        for word in words:
            #score += math.log(self.word_prob(word, category))
            score += math.log( (self.cfd[category][word]+1) / (self.category_dict[category] + len(self.words)) )
        return score
    
    # 예측하기 --- (※5)
    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize 
        words = self.split(text)
        score_list = []
        for category in self.category_dict.keys():
            score = self.score(words, category)
            score_list.append((category, score))
            if score > max_score:
                max_score = score
                best_category = category
        return best_category, score_list

    # 카테고리 내부의 단어 출현 비율 계산 --- (※6)
    def word_prob(self, word, category):
        n = self.get_word_count(word, category) + 1 # ---(※6a)
        d = sum(self.word_dict[category].values()) + len(self.words)
        return n / d

def content_prep(text, stopwords):
    #stopwords = nltk.corpus.stopwords.words('english')
    content = []
    wnl = nltk.WordNetLemmatizer()
    
    for c,w in text:
        #word = wnl.lemmatize(w)
        word = w
        #if(word!=w):
        #    print(w,word)
        if word.lower() not in stopwords and re.search('[A-Za-z0-9]$', word):
        #if word.lower() not in stopwords:
            content+={(c, word)}
        #content = [w for w in word if w.lower() not in stopwords]
    #for i in text:
    #    content = [w for w in text if w.lower() not in stopwords]
    return content

In [411]:
genre_word = [(genre,word) 
              for genre in ['romance','science_fiction','news']
              for word in  brown.words(categories=genre)]
              #for content_fraction(word,stopwords) in brown.words(categories=genre)]

In [413]:
myst_words = stopwords.words('english')
#mylist = ['-',',','.','"','\'',';']
#myst_words += mylist
#myst_words
fraction_genre_word = content_prep(genre_word,myst_words)
#fraction_genre_word = genre_word

In [395]:
bf = BayesianFilter()
bf.fit_list(fraction_genre_word)

In [396]:
#cfd = nltk.ConditionalFreqDist(fraction_genre_word)
#cfd['news']['The']
#for category in cfd.conditions():
#    for word in cfd[category]:
#        print(word)
#        print(cfd[category][word])
#cfd['science_fiction']

In [397]:
#bf = BayesianFilter()
#bf.fit(cfd)
#print(bf.word_dict)
#print(bf.category_dict)
fraction_genre_word[:10]

[('romance', 'neither'),
 ('romance', 'liked'),
 ('romance', 'disliked'),
 ('romance', 'Old'),
 ('romance', 'Man'),
 ('romance', 'could'),
 ('romance', 'broken'),
 ('romance', 'bell'),
 ('romance', 'church'),
 ('romance', 'tower')]

In [398]:
# 예측
accuracy = 0.0
t_count = 0
wnl = nltk.WordNetLemmatizer()
for c,w in fraction_genre_word:
    pre, scorelist = bf.predict(w)
    if(pre == c):
        t_count += 1
    #print('s word',w)
    #print('s cate',c)
    #print("결과 =", pre)
    #print("결과 =", scorelist)
    #print()
    
print(round(t_count/len(fraction_genre_word),3)*100,'%')

76.9 %


In [414]:
tokens = "The little yellow dog barked at the Persian cat".split()
print(tokens)
tags_en = nltk.pos_tag(tokens)

['The', 'little', 'yellow', 'dog', 'barked', 'at', 'the', 'Persian', 'cat']


In [390]:
print(tags_en)

[('The', 'DT'), ('little', 'JJ'), ('yellow', 'JJ'), ('dog', 'NN'), ('barked', 'VBD'), ('at', 'IN'), ('the', 'DT'), ('Persian', 'JJ'), ('cat', 'NN')]
